**SC 500M Instructions MOP**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
baseline_4cores_out_path = '../../prac_study/TimingChannel/SC/OriginalTiming'

df = pd.DataFrame(columns=["mitigation", "workload", "WS"])
mitigation_list = ["Baseline", "1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', 
                   '2RFM-1tREFI', '4RFM-1tREFI']
for mitigation in mitigation_list:
    result_path = baseline_4cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])

        workload = "_".join(result_filename.split("_")[2:])

        ipc_0 = 0
        cycle_0 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_abo=0
        num_tREFI_period=0
        num_tREFW_period=0
        num_rd_reqs=0
        num_wr_reqs=0
        for line in result_file.readlines():
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" prac_num_recovery" in line):
                num_abo = int(line.split(" ")[-1])
            if (" num_refresh_command_0" in line):
                num_tREFI_period = int(line.split(" ")[-1])
            if (" num_refresh_window_0" in line):
                num_tREFW_period = int(line.split(" ")[-1])            
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])

        if (cycle_0 == 0):
            continue
        if (cycle_0 == 0 ):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))

        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'WS': [ipc_0],
        })
        df = pd.concat([df, new_row], ignore_index=True)


df_ws = df.pivot(index=['workload', 'Channel', 'interface'], columns=['mitigation'], values='WS').reset_index()
df_ws[['workload', 'Channel','interface', 'Baseline']].to_csv('../stats/SC_500M_DDR5_8000_Original_Baseline.csv', index=False)

for mitigation in set(mitigation_list) - set(['Baseline']):
     df_ws[mitigation] = df_ws[mitigation] / df_ws['Baseline']
df_ws.drop(columns=['Baseline'], inplace=True)

# Define benchmark suites and their corresponding workloads
benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 23
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

#
mitigation_list = ["1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', 
                   '2RFM-1tREFI', '4RFM-1tREFI']
# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            for suite_name, workloads in benchmark_suites.items():
                suite_df = df[(df['workload'].isin(workloads)) & (df['Channel'] == Channel) & (df['interface'] == interface)]
                if not suite_df.empty:
                    geomeans = {}
                    
                    # Dynamically calculate geometric means for each mitigation
                    for mitigation in mitigation_list:
                        if mitigation in suite_df.columns:  # Ensure the column exists
                            geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                    
                    # Create a new row
                    geomean_row = {'Channel': Channel, 'interface': interface, 'workload': suite_name, **geomeans}
                    geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            Channel_interface_df = df[(df['Channel'] == Channel) & (df['interface'] == interface)]
            geomean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'Channel': Channel, 'interface': interface, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)
# Call function to calculate and merge geometric means
geomean_df = add_geomean_rows(df_ws)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df.to_csv('./../stats/SC_500M_Timing_Based_RFM.csv', index=False)

          workload  Channel  interface  1RFM-1tREFI  1RFM-2tREFI  1RFM-4tREFI  \
0        401.bzip2      1.0     8000.0     0.961749     0.981761     0.990550   
1          403.gcc      1.0     8000.0     0.990611     0.996017     0.997740   
2          429.mcf      1.0     8000.0     0.893537     0.946820     0.973584   
3         433.milc      1.0     8000.0     0.896146     0.948166     0.974270   
4       434.zeusmp      1.0     8000.0     0.877834     0.938773     0.969148   
..             ...      ...        ...          ...          ...          ...   
59         TPC (4)      1.0     8000.0     0.917320     0.958564     0.979780   
60      Hadoop (3)      1.0     8000.0     0.914041     0.957658     0.978984   
61  MediaBench (3)      1.0     8000.0     0.951537     0.976708     0.988594   
62        YCSB (6)      1.0     8000.0     0.900282     0.949754     0.974778   
63        All (57)      1.0     8000.0     0.928146     0.964548     0.982457   

    2RFM-1tREFI  4RFM-1tREF

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
baseline_4cores_out_path = '../../prac_study/TimingChannel/SC/PRACTiming'

df = pd.DataFrame(columns=["mitigation", "workload", "WS"])
mitigation_list = ["Baseline", "1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', 
                   '2RFM-1tREFI', '4RFM-1tREFI']
for mitigation in mitigation_list:
    result_path = baseline_4cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])

        workload = "_".join(result_filename.split("_")[2:])

        ipc_0 = 0
        cycle_0 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_abo=0
        num_tREFI_period=0
        num_tREFW_period=0
        num_rd_reqs=0
        num_wr_reqs=0
        for line in result_file.readlines():
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" prac_num_recovery" in line):
                num_abo = int(line.split(" ")[-1])
            if (" num_refresh_command_0" in line):
                num_tREFI_period = int(line.split(" ")[-1])
            if (" num_refresh_window_0" in line):
                num_tREFW_period = int(line.split(" ")[-1])            
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])

        if (cycle_0 == 0):
            continue
        if (cycle_0 == 0 ):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))

        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'WS': [ipc_0],
        })
        df = pd.concat([df, new_row], ignore_index=True)

df_ws = df.pivot(index=['workload', 'Channel', 'interface'], columns=['mitigation'], values='WS').reset_index()
df_ws[['workload', 'Channel','interface', 'Baseline']].to_csv('../stats/SC_500M_DDR5_8000_PRAC_Baseline.csv', index=False)

for mitigation in set(mitigation_list) - set(['Baseline']):
     df_ws[mitigation] = df_ws[mitigation] / df_ws['Baseline']
df_ws.drop(columns=['Baseline'], inplace=True)

# Define benchmark suites and their corresponding workloads
benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 23
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

#
mitigation_list = ["1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', 
                   '2RFM-1tREFI', '4RFM-1tREFI']
# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            for suite_name, workloads in benchmark_suites.items():
                suite_df = df[(df['workload'].isin(workloads)) & (df['Channel'] == Channel) & (df['interface'] == interface)]
                if not suite_df.empty:
                    geomeans = {}
                    
                    # Dynamically calculate geometric means for each mitigation
                    for mitigation in mitigation_list:
                        if mitigation in suite_df.columns:  # Ensure the column exists
                            geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                    
                    # Create a new row
                    geomean_row = {'Channel': Channel, 'interface': interface, 'workload': suite_name, **geomeans}
                    geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            Channel_interface_df = df[(df['Channel'] == Channel) & (df['interface'] == interface)]
            geomean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'Channel': Channel, 'interface': interface, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)
# Call function to calculate and merge geometric means
geomean_df = add_geomean_rows(df_ws)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df.to_csv('./../stats/SC_500M_Timing_Based_RFM_PRAC_Timing.csv', index=False)

          workload  Channel  interface  1RFM-1tREFI  1RFM-2tREFI  1RFM-4tREFI  \
0        401.bzip2      1.0     8000.0     0.969892     0.985358     0.992436   
1          403.gcc      1.0     8000.0     0.996713     0.999167     0.999464   
2          429.mcf      1.0     8000.0     0.925416     0.962599     0.981274   
3         433.milc      1.0     8000.0     0.931943     0.965892     0.983253   
4       434.zeusmp      1.0     8000.0     0.900488     0.949822     0.975212   
..             ...      ...        ...          ...          ...          ...   
59         TPC (4)      1.0     8000.0     0.936132     0.968988     0.984172   
60      Hadoop (3)      1.0     8000.0     0.926400     0.963683     0.981986   
61  MediaBench (3)      1.0     8000.0     0.958530     0.979662     0.990032   
62        YCSB (6)      1.0     8000.0     0.920888     0.960317     0.980441   
63        All (57)      1.0     8000.0     0.943388     0.972088     0.985996   

    2RFM-1tREFI  4RFM-1tREF

**4-Core, MOP, 500M, Open Page, Varying Channel**

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/TimingChannel/4cores_results/OriginalTiming'

df = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["Baseline", "1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', 
                   '2RFM-1tREFI', '4RFM-1tREFI']
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])
        workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))
        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'wl0': [w0],
            'wl1': [w1],
            'wl2': [w2],
            'wl3': [w3],
            'ipc0': [ipc_0],
            'ipc1': [ipc_1],
            'ipc2': [ipc_2],
            'ipc3': [ipc_3],
        })
        df = pd.concat([df, new_row], ignore_index=True)

df_sc_ipc = pd.read_csv('../stats/SC_500M_DDR5_8000_Original_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['interface'] == 8000)]
# df_sc_ipc = df_sc_ipc.drop(columns=['interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl0'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl1'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl2'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl3'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup

df['normalzied_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalzied_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalzied_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalzied_ipc3'] = df['ipc3'] / df['ipc_wl3']

df['WS'] = df[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)

df_closed_cap1_ws = df[['mitigation', 'workload', 'Channel', 'WS']]

df_closed_cap1_ws_pivot = df_closed_cap1_ws.pivot(index=['workload', 'Channel'], columns=['mitigation'], values='WS').reset_index()
for mitigation in set(mitigation_list) - set(['Baseline']):
     df_closed_cap1_ws_pivot[mitigation] = df_closed_cap1_ws_pivot[mitigation] / df_closed_cap1_ws_pivot['Baseline']
df_closed_cap1_ws_pivot.drop(columns=['Baseline'], inplace=True)
##### Calculate the Geomean for each workload type
# Define benchmark suites and their corresponding workloads ranges
benchmark_types = {
    'HHHH (15)': list(range(0, 15)),   # From Mix0-Mix14
    'MMMM (15)': list(range(15, 30)),  # From Mix15-Mix29
    'LLLL (15)': list(range(30, 45)),  # From Mix30-Mix44
    'HHMM (15)': list(range(45, 60)),  # From Mix45-Mix59
    'HHLL (15)': list(range(60, 75)),  # From Mix60-Mix74
    'MMLL (15)': list(range(75, 90)),  # From Mix75-Mix89
}

# DataFrame Example (you already have your df)
# Assuming your column of interest is 'PRAC_WO_Mitigation-ClosedCap1'

# Add a new column to assign each Mix to the appropriate suite
def assign_benchmark_types(mix_index):
    for suite, mix_range in benchmark_types.items():
        if mix_index in mix_range:
            return suite
    return None

# Assuming 'workload' has values like 'MIX0', 'MIX1', etc., you can extract the index
df_closed_cap1_ws_pivot['Mix_index'] = df_closed_cap1_ws_pivot['workload'].str.extract(r'(\d+)').astype(int)  # Extract Mix number
df_closed_cap1_ws_pivot['Benchmark_Types'] = df_closed_cap1_ws_pivot['Mix_index'].apply(assign_benchmark_types)  # Assign benchmark suite

# Function to calculate the geometric mean
def geom_mean(series):
    return np.exp(np.log(series).mean())

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for suite_name, mix_indices in benchmark_types.items():
            # Create a list of corresponding workload names (e.g., MIX0, MIX1)
            workloads = [f'MIX{i}' for i in mix_indices]
            suite_df = df[df['workload'].isin(workloads) & (df['Channel'] == Channel)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'Channel': Channel, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        Channel_df = df[df['Channel'] == Channel]
        geomean_values = {}
        
        # Calculate geometric means for each mitigation in the list
        for mitigation in mitigation_list:
            if mitigation in Channel_df.columns:  # Ensure the column exists
                geomean_values[mitigation] = calculate_geometric_mean(Channel_df[mitigation])

        # Create a new row for the combined results
        geomean_row = {'Channel': Channel, 'workload': 'All (90)', **geomean_values}
        geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

mitigation_list = ["1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', '2RFM-1tREFI', '4RFM-1tREFI']

geomean_df = add_geomean_rows(df_closed_cap1_ws_pivot)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df[['workload', 'Channel', "Benchmark_Types", "1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', '2RFM-1tREFI', '4RFM-1tREFI']].to_csv('../stats/4cores_OPTMC_DDR5_8000_500M_Heterogeneous_Timing_Based_RFM_Original.csv', index=False)


     workload  Channel  1RFM-1tREFI  1RFM-2tREFI  1RFM-4tREFI  2RFM-1tREFI  \
0        MIX0      1.0     0.880229     0.940200     0.971027     0.758118   
1        MIX1      1.0     0.872512     0.935833     0.956798     0.755463   
2       MIX10      1.0     0.879150     0.939839     0.970387     0.757806   
3       MIX11      1.0     0.880567     0.934922     0.966918     0.752932   
4       MIX12      1.0     0.898994     0.951332     0.975193     0.800485   
..        ...      ...          ...          ...          ...          ...   
92  LLLL (15)      1.0     0.961435     0.980751     0.990163     0.921014   
93  HHMM (15)      1.0     0.888772     0.944122     0.971781     0.776921   
94  HHLL (15)      1.0     0.922418     0.961600     0.980955     0.843655   
95  MMLL (15)      1.0     0.926868     0.963279     0.980894     0.853265   
96   All (90)      1.0     0.912891     0.956559     0.977938     0.824076   

    4RFM-1tREFI  Mix_index Benchmark_Types  
0      0.515321   

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/TimingChannel/4cores_results/PRACTiming'

df = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["Baseline", "1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', 
                   '2RFM-1tREFI', '4RFM-1tREFI']
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])
        workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))
        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'wl0': [w0],
            'wl1': [w1],
            'wl2': [w2],
            'wl3': [w3],
            'ipc0': [ipc_0],
            'ipc1': [ipc_1],
            'ipc2': [ipc_2],
            'ipc3': [ipc_3],
        })
        df = pd.concat([df, new_row], ignore_index=True)

df_sc_ipc = pd.read_csv('../stats/SC_500M_DDR5_8000_PRAC_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['interface'] == 8000)]
# df_sc_ipc = df_sc_ipc.drop(columns=['interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl0'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl1'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl2'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl3'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup

df['normalzied_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalzied_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalzied_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalzied_ipc3'] = df['ipc3'] / df['ipc_wl3']

df['WS'] = df[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)

df_closed_cap1_ws = df[['mitigation', 'workload', 'Channel', 'WS']]

df_closed_cap1_ws_pivot = df_closed_cap1_ws.pivot(index=['workload', 'Channel'], columns=['mitigation'], values='WS').reset_index()
for mitigation in set(mitigation_list) - set(['Baseline']):
     df_closed_cap1_ws_pivot[mitigation] = df_closed_cap1_ws_pivot[mitigation] / df_closed_cap1_ws_pivot['Baseline']
df_closed_cap1_ws_pivot.drop(columns=['Baseline'], inplace=True)
##### Calculate the Geomean for each workload type
# Define benchmark suites and their corresponding workloads ranges
benchmark_types = {
    'HHHH (15)': list(range(0, 15)),   # From Mix0-Mix14
    'MMMM (15)': list(range(15, 30)),  # From Mix15-Mix29
    'LLLL (15)': list(range(30, 45)),  # From Mix30-Mix44
    'HHMM (15)': list(range(45, 60)),  # From Mix45-Mix59
    'HHLL (15)': list(range(60, 75)),  # From Mix60-Mix74
    'MMLL (15)': list(range(75, 90)),  # From Mix75-Mix89
}

# DataFrame Example (you already have your df)
# Assuming your column of interest is 'PRAC_WO_Mitigation-ClosedCap1'

# Add a new column to assign each Mix to the appropriate suite
def assign_benchmark_types(mix_index):
    for suite, mix_range in benchmark_types.items():
        if mix_index in mix_range:
            return suite
    return None

# Assuming 'workload' has values like 'MIX0', 'MIX1', etc., you can extract the index
df_closed_cap1_ws_pivot['Mix_index'] = df_closed_cap1_ws_pivot['workload'].str.extract(r'(\d+)').astype(int)  # Extract Mix number
df_closed_cap1_ws_pivot['Benchmark_Types'] = df_closed_cap1_ws_pivot['Mix_index'].apply(assign_benchmark_types)  # Assign benchmark suite

# Function to calculate the geometric mean
def geom_mean(series):
    return np.exp(np.log(series).mean())

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for suite_name, mix_indices in benchmark_types.items():
            # Create a list of corresponding workload names (e.g., MIX0, MIX1)
            workloads = [f'MIX{i}' for i in mix_indices]
            suite_df = df[df['workload'].isin(workloads) & (df['Channel'] == Channel)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'Channel': Channel, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        Channel_df = df[df['Channel'] == Channel]
        geomean_values = {}
        
        # Calculate geometric means for each mitigation in the list
        for mitigation in mitigation_list:
            if mitigation in Channel_df.columns:  # Ensure the column exists
                geomean_values[mitigation] = calculate_geometric_mean(Channel_df[mitigation])

        # Create a new row for the combined results
        geomean_row = {'Channel': Channel, 'workload': 'All (90)', **geomean_values}
        geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

mitigation_list = ["1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', '2RFM-1tREFI', '4RFM-1tREFI']

geomean_df = add_geomean_rows(df_closed_cap1_ws_pivot)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df[['workload', 'Channel', "Benchmark_Types", "1RFM-4tREFI", '1RFM-2tREFI', '1RFM-1tREFI', '2RFM-1tREFI', '4RFM-1tREFI']].to_csv('../stats/4cores_OPTMC_DDR5_8000_500M_Heterogeneous_Timing_Based_RFM_PRAC.csv', index=False)
